<a href="https://colab.research.google.com/github/MBilalSharif/RAG-based-CV-Reader/blob/main/CV_Reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -q langchain langchain-community chromadb sentence-transformers pypdf google-generativeai
print("Installed!")

✅ Installed!


In [26]:
from google.colab import files

print("Upload your CV (PDF or TXT): ")
uploaded = files.upload()
cv_filename = list(uploaded.keys())[0]
print(f"Uploaded: {cv_filename}")

📄 Upload your CV (PDF or TXT): 


Saving M Bilal Sharif.pdf to M Bilal Sharif (2).pdf
✅ Uploaded: M Bilal Sharif (2).pdf


In [27]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_core.documents import Document

# Load based on file type
if cv_filename.endswith('.pdf'):
    loader = PyPDFLoader(cv_filename)
    documents = loader.load()
elif cv_filename.endswith('.txt'):
    loader = TextLoader(cv_filename)
    documents = loader.load()
else:
    with open(cv_filename, 'r') as f:
        documents = [Document(page_content=f.read())]

print(f"Loaded {len(documents)} document(s)")
print(f"Total characters: {sum(len(doc.page_content) for doc in documents)}")

✅ Loaded 1 document(s)
📝 Total characters: 4496


In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

✅ Created 36 chunks


In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Create embeddings (convert text to numbers)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./cv_vectorstore"
)
print("Vector database created!")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


🔄 Creating vector database...
✅ Vector database created!


In [30]:
import google.generativeai as genai
from getpass import getpass


api_key = getpass("🔑 Paste your Google AI API key: ")
genai.configure(api_key=api_key)

model = genai.GenerativeModel(
    model_name='gemini-pro',
    generation_config={
        'temperature': 0.3,
        'max_output_tokens': 500,
    }
)

print("Gemini ready!")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


🔑 Paste your Google AI API key: ··········
✅ Gemini ready!


In [37]:
def ask_cv_question(question, k=5):
    """Ask a question about your CV using RAG"""


    print(f"🔍 Searching for relevant info...")
    relevant_docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    print(f"✅ Found {len(relevant_docs)} relevant sections\n")

    # GENERATION with structured format
    prompt = f"""You must respond with ONLY the answer, nothing else.

CV Information:
{context}

Question: {question}

Rules:
1. Extract the exact answer from the CV
2. Do NOT add any preamble like "The answer is" or "According to"
3. Do NOT add explanations
4. Just state the fact directly

Example:
Question: What is the person's email?
Bad: "The person's email address is john@example.com"
Good: "john@example.com"

Question: What is the person's name?
Bad: "According to the CV, the person's name is John Doe"
Good: "John Doe"

Now answer the actual question above with just the direct answer:"""

    print("🤖 Generating answer...\n")

    try:
        response = model.generate_content(prompt)
        answer = response.text.strip()
    except Exception as e:
        answer = f"Error: {str(e)}"

    # Display result
    print("="*60)
    print(f"❓ Question: {question}")
    print("="*60)
    print(f"💡 Answer: {answer}")
    print("="*60)

    return answer

print("✅ RAG function ready!")

✅ RAG function ready!


In [38]:
ask_cv_question("What is the name of Applicant?")

🔍 Searching for relevant info...
✅ Found 5 relevant sections

🤖 Generating answer...



❓ Question: What is the name of Applicant?
💡 Answer: Error: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.


'Error: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.'